In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy import stats
import pandas as pd
import tensorflow as tf
import numpy as np

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
application_df['INCOME_AMT'] = application_df['INCOME_AMT'].replace('0', '1-1')
application_df['INCOME_AMT'] = application_df['INCOME_AMT'].replace('5M-10M', '5000000-1000000')
application_df['INCOME_AMT'] = application_df['INCOME_AMT'].replace('50M+', '50000000-60000000')
application_df['INCOME_AMT'] = application_df['INCOME_AMT'].replace('10M-50M', '35000000-35000000')
application_df['INCOME_AMT'] = application_df['INCOME_AMT'].replace('1M-5M', '3500000-3500000')
application_df['INCOME_AMT'].sort_values(ascending=False).head(325)

21641    50000000-60000000
11824    50000000-60000000
32385    50000000-60000000
32384    50000000-60000000
24257    50000000-60000000
               ...        
23233      5000000-1000000
33052      5000000-1000000
23821      5000000-1000000
33137      5000000-1000000
21455    35000000-35000000
Name: INCOME_AMT, Length: 325, dtype: object

In [3]:
def incomeaskratio(income): 
    high_low = income.split('-')
    average = (int(high_low[0]) + int(high_low[1]))/2
    return average
        
        
application_df['ASK_INCOME_RATIO'] = application_df['ASK_AMT']/application_df['INCOME_AMT'].apply(lambda x: incomeaskratio(x))


In [4]:
application_df['INCOME_AMT'] = application_df['INCOME_AMT'].apply(lambda x: incomeaskratio(x))

In [5]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns = ['EIN','STATUS', 'SPECIAL_CONSIDERATIONS'], inplace = True)


In [6]:
application_df = application_df[(np.abs(stats.zscore(application_df['ASK_AMT'])) < 3)].dropna()


In [7]:
application_df['USE_CASE'].value_counts()

Preservation     28065
ProductDev        5668
CommunityServ      384
Heathcare          126
Other                3
Name: USE_CASE, dtype: int64

In [8]:
use_cases_to_replace = ['CommunityServ', 'Heathcare']

# Replace in dataframe
for use in use_cases_to_replace:
    application_df['USE_CASE'] = application_df['USE_CASE'].replace(use,"Other")

# Check to make sure binning was successful
application_df['USE_CASE'].value_counts()

Preservation    28065
ProductDev       5668
Other             513
Name: USE_CASE, dtype: int64

In [9]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

count = application_df.APPLICATION_TYPE.value_counts()
application_types_to_replace = list(count[count < 200].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       26987
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      273
Name: APPLICATION_TYPE, dtype: int64

In [10]:
count = application_df.CLASSIFICATION.value_counts()
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(count[count < 800].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17291
C2000     6064
C1200     4831
Other     2259
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [11]:
count = application_df.NAME.value_counts()
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
names_to_replace = list(count[count < 50].index)

# Replace in dataframe
for name in names_to_replace:
    application_df['NAME'] = application_df['NAME'].replace(name,"OTHER")
    
# Check to make sure binning was successful
application_df['NAME'].value_counts()

OTHER                                                                 24492
PARENT BOOSTER USA INC                                                 1260
TOPS CLUB INC                                                           765
UNITED STATES BOWLING CONGRESS INC                                      700
WASHINGTON STATE UNIVERSITY                                             492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                         408
PTA TEXAS CONGRESS                                                      368
SOROPTIMIST INTERNATIONAL OF THE AMERICAS INC                           331
ALPHA PHI SIGMA                                                         313
TOASTMASTERS INTERNATIONAL                                              293
MOST WORSHIPFUL STRINGER FREE AND ACCEPTED MASONS                       287
LITTLE LEAGUE BASEBALL INC                                              277
INTERNATIONAL ASSOCIATION OF LIONS CLUBS                                266
MOMS CLUB   

In [12]:
dummy_df = pd.get_dummies(application_df)

In [13]:
# Split our preprocessed data into our features and target arrays
features = dummy_df.drop(columns = ['IS_SUCCESSFUL']).values
target = dummy_df['IS_SUCCESSFUL'].values
# Split the preprocessed data into a training and testing dataset

X_train, X_test, Y_train, Y_test = train_test_split(features, target)

In [14]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_train[0]

array([1.e+00, 5.e+03, 5.e+03, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00,
       0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00,
       0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00,
       0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 1.e+00, 0.e+00,
       0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00,
       0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00,
       0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00,
       0.e+00, 0.e+00, 1.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00,
       1.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00,
       1.e+00, 0.e+00, 0.e+00, 0.e+00, 1.e+00, 0.e+00, 0.e+00, 0.e+00,
       0.e+00, 0.e+00, 1.e+00])

In [15]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
inputs = len(X_train[0])
layer1 = 100
layer2 = 40
layer3 = 30
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units = layer1, input_dim=inputs, activation = "swish")
)

# Second hidden layer
nn.add(
    tf.keras.layers.Dense(units = layer2, input_dim=layer1, activation = "swish")
)

nn.add(
    tf.keras.layers.Dense(units = layer3, input_dim=layer2, activation = "swish")
)

# Output layer
nn.add(
    tf.keras.layers.Dense(units = 1, input_dim=layer3, activation = "sigmoid")
)
# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               8400      
_________________________________________________________________
dense_1 (Dense)              (None, 40)                4040      
_________________________________________________________________
dense_2 (Dense)              (None, 30)                1230      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 31        
Total params: 13,701
Trainable params: 13,701
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics="accuracy")

In [19]:
# Train the model
fit_model = nn.fit(X_train_scaled, Y_train, epochs=100)

Epoch 1/100
803/803 [==============================] - 1s 663us/step - loss: 0.4832 - accuracy: 0.7613
Epoch 2/100
803/803 [==============================] - 1s 665us/step - loss: 0.4801 - accuracy: 0.7635
Epoch 3/100
803/803 [==============================] - 1s 663us/step - loss: 0.4795 - accuracy: 0.7633
Epoch 4/100
803/803 [==============================] - 1s 660us/step - loss: 0.4772 - accuracy: 0.7657
Epoch 5/100
803/803 [==============================] - 1s 663us/step - loss: 0.4769 - accuracy: 0.7650
Epoch 6/100
803/803 [==============================] - 1s 665us/step - loss: 0.4764 - accuracy: 0.7657
Epoch 7/100
803/803 [==============================] - 1s 668us/step - loss: 0.4754 - accuracy: 0.7659
Epoch 8/100
803/803 [==============================] - 1s 673us/step - loss: 0.4750 - accuracy: 0.7662
Epoch 9/100
803/803 [==============================] - 1s 677us/step - loss: 0.4742 - accuracy: 0.7669
Epoch 10/100
803/803 [==============================] - 1s 673us/step - l

803/803 [==============================] - 1s 675us/step - loss: 0.4624 - accuracy: 0.7743
Epoch 80/100
803/803 [==============================] - 1s 677us/step - loss: 0.4627 - accuracy: 0.7743
Epoch 81/100
803/803 [==============================] - 1s 673us/step - loss: 0.4626 - accuracy: 0.7731
Epoch 82/100
803/803 [==============================] - 1s 683us/step - loss: 0.4623 - accuracy: 0.7741
Epoch 83/100
803/803 [==============================] - 1s 673us/step - loss: 0.4620 - accuracy: 0.7747
Epoch 84/100
803/803 [==============================] - 1s 673us/step - loss: 0.4623 - accuracy: 0.7743
Epoch 85/100
803/803 [==============================] - 1s 670us/step - loss: 0.4618 - accuracy: 0.7741
Epoch 86/100
803/803 [==============================] - 1s 678us/step - loss: 0.4620 - accuracy: 0.7738
Epoch 87/100
803/803 [==============================] - 1s 667us/step - loss: 0.4621 - accuracy: 0.7741
Epoch 88/100
803/803 [==============================] - 1s 672us/step - loss:

In [20]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,Y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5072 - accuracy: 0.7624
Loss: 0.5071990489959717, Accuracy: 0.7624386548995972


In [21]:
nn.save('AlphabetSoupCharity_Optimzation.h5')